## Setup

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# For emptying trash after run
#from google.colab import auth
#auth.authenticate_user()
#from googleapiclient.discovery import build
#drive_service = build('drive', 'v3')
#drive_service.files().emptyTrash().execute()

In [3]:
folder_name = "/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/"
#folder_name = "/content/drive/MyDrive/persuasion_technique_detection/"

## Imports

In [4]:
#!pip install transformers datasets wandb evaluate accelerate -qU sklearn_hierarchical_classification sentencepiece

In [5]:
import gc
import re
import json
import numpy as np
import pandas as pd
import random
import torch
import subprocess
import json
import warnings
import shutil
import os

In [6]:
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
from datasets import Dataset,load_dataset,DatasetDict,concatenate_datasets
import datasets
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from datasets import concatenate_datasets
from transformers import Trainer
from transformers import AutoTokenizer, DataCollatorWithPadding

In [7]:

AVAIL_GPUS = 0
if torch.cuda.is_available():
    device = torch.device("cuda")
    AVAIL_GPUS = torch.cuda.device_count()
    print(f'There are {AVAIL_GPUS} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3080 Ti


In [8]:
model_name="Random_Forest"
summary_dir_path = folder_name + "subtask1/transformer/summaries/ensembling/summary_ensembling_hierarchical_test_pred" + model_name.replace("/","_") + "/"
train_set = 'train_dev'

## Base Models

#### Data

In [9]:
# Initialize a counter for empty label samples
empty_label_counter = [0]

def filter_every_second_empty_label(sample):
    if len(sample['labels']) == 0:
        empty_label_counter[0] += 1

        return empty_label_counter[0] % 2 != 0
    return True

In [10]:
val_path=folder_name+"data/subtask1/validation.json"
train_path = folder_name+"data/subtask1/train.json"
dev_path=folder_name+"data/subtask1/dev_subtask1_en.json"
train_dev_path = folder_name+"data/subtask1/train_dev.json"
test_path=folder_name+"data/subtask1/en_subtask1_test_unlabeled.json"

train_added_ptc = folder_name+"data/ptc/ptc_added_train.json"
train_val_path = folder_name+"data/subtask1/train_val.json"

#dataset_memes_2024_files = {"train": train_added_ptc,"validation": val_path}
dataset_memes_2024_files = {"train": train_path,"validation": val_path, "train_val": train_val_path}
train_files={"train":train_path}
val_files={"val":val_path}
test_files={"test":test_path}
train_dev_files={"train_dev":train_dev_path}
train_val_files={"train_val":train_val_path}

dataset_memes_2024 = load_dataset("json",data_files=dataset_memes_2024_files)

empty_labels = 1
if empty_labels == 0:
  dataset_memes_2024 = dataset_memes_2024.filter(lambda x : len(x['labels']) != 0)
if empty_labels == 0.5:
  dataset_memes_2024 = dataset_memes_2024.filter(filter_every_second_empty_label)

dataset_test=load_dataset("json",data_files=test_files)
dataset_val=load_dataset("json",data_files=val_files)
dataset_train=load_dataset("json", data_files=train_files)
dataset_train_val=load_dataset("json", data_files=train_val_files)
dataset_train_dev=load_dataset("json", data_files=train_dev_files)

### Hierarchical

Login to WandB

In [11]:
import wandb
import os

#wandb.login(relogin=True)
wandb.login()

# setup wandb environment variables
os.environ['WANDB_PROJECT'] = "subtask1_transformer_encoder_classification"
os.environ['WANDB_ENTITY'] = "tumnlp"
os.environ["WANDB_LOG_MODEL"]= "end"

wandb: Currently logged in as: mahmudfami (tumnlp). Use `wandb login --relogin` to force relogin


In [12]:
checkpoint_hierarchical_1="vinai/bertweet-large"
#checkpoint_hierarchical = "microsoft/deberta-v3-large"

In [13]:
persuasion_unique=[["Ethos","Pathos","Logos"]]
ethos_unique=[["Ad Hominem","Bandwagon","Appeal to authority","Glittering generalities (Virtue)"]]
pathos_unique=[["Exaggeration/Minimisation","Loaded Language","Flag-waving","Appeal to fear/prejudice"]]
logos_unique=[["Justification","Reasoning","Repetition","Obfuscation, Intentional vagueness, Confusion"]]
ad_hominem_unique=[["Name calling/Labeling","Doubt","Smears","Reductio ad hitlerum","Whataboutism"]]
justification_unique=[["Bandwagon","Appeal to authority","Flag-waving","Appeal to fear/prejudice","Slogans"]]
reasoning_unique=[["Distraction","Simplification"]]
distraction_unique=[["Misrepresentation of Someone's Position (Straw Man)","Presenting Irrelevant Data (Red Herring)","Whataboutism"]]
simplification_unique=[["Causal Oversimplification","Black-and-white Fallacy/Dictatorship","Thought-terminating cliché"]]
persuasion_or_not_unique=[["Not Persuasion","Persuasion"]]

In [14]:
mlb_persuasion = MultiLabelBinarizer()
mlb_persuasion.fit(persuasion_unique)

mlb_ethos = MultiLabelBinarizer()
mlb_ethos.fit(ethos_unique)

mlb_pathos = MultiLabelBinarizer()
mlb_pathos.fit(pathos_unique)

mlb_logos = MultiLabelBinarizer()
mlb_logos.fit(logos_unique)

mlb_ad_hominem = MultiLabelBinarizer()
mlb_ad_hominem.fit(ad_hominem_unique)

mlb_justification = MultiLabelBinarizer()
mlb_justification.fit(justification_unique)

mlb_reasoning=MultiLabelBinarizer()
mlb_reasoning.fit(reasoning_unique)

mlb_distraction=MultiLabelBinarizer()
mlb_distraction.fit(distraction_unique)

mlb_simplification=MultiLabelBinarizer()
mlb_simplification.fit(simplification_unique)

lb_persuasion_or_not=LabelEncoder()
lb_persuasion_or_not.fit(persuasion_or_not_unique[0])


LabelEncoder()

In [15]:
techniques_nodes = [['Black-and-white Fallacy/Dictatorship', 'Loaded Language',
       'Glittering generalities (Virtue)', 'Thought-terminating cliché',
       'Whataboutism', 'Slogans', 'Causal Oversimplification', 'Smears',
       'Name calling/Labeling', 'Appeal to authority',
       'Exaggeration/Minimisation', 'Repetition', 'Flag-waving',
       'Appeal to fear/prejudice', 'Reductio ad hitlerum', 'Doubt',
       "Misrepresentation of Someone's Position (Straw Man)",
       'Obfuscation, Intentional vagueness, Confusion', 'Bandwagon',
       'Presenting Irrelevant Data (Red Herring)', 'Persuasion', 'Ethos', 'Pathos', 'Logos', 'Ad Hominem', 'Justification','Reasoning', 'Simplification', 'Distraction']] 

In [16]:
mlb_extended_nodes = MultiLabelBinarizer()
mlb_extended_nodes.fit(techniques_nodes)

MultiLabelBinarizer()

In [17]:
tokenizer_hierarchical_1 = AutoTokenizer.from_pretrained(checkpoint_hierarchical_1)
data_collator_hierarchical_1 = DataCollatorWithPadding(tokenizer=tokenizer_hierarchical_1)
def tokenize_hierarchical_1(examples):
    encoding = tokenizer_hierarchical_1(examples["text"], truncation=True ,padding='max_length', max_length=512)
    return encoding

In [18]:
def get_preds_hierarchical(binarizer,predicted_logits,threshold=0.5):

  predicted_logits_tensor = torch.from_numpy(predicted_logits)

  if isinstance(binarizer,LabelEncoder):
    softmax=torch.nn.Softmax(dim=1)
    full_probs=softmax(predicted_logits_tensor)

    probs,predictions=torch.max(full_probs,dim=1)
    predictions=[[p] for p in binarizer.inverse_transform(predictions)]
    probs = [[value.item()] for value in probs]
    res=[(x, y) for x, y in zip(predictions, probs)]
    return res
  else:
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(predicted_logits_tensor)
    preds=(probs>threshold).int()

    mask = (preds == 1)

    # Use the mask to select the corresponding probabilities
    selected_probs_list = []

    # Iterate over rows and select probabilities using the mask
    for i in range(preds.size(0)):
        selected_probs_row = probs[i][mask[i]].tolist()
        selected_probs_list.append(selected_probs_row)

    names=binarizer.inverse_transform(preds)
    res=[(list(x), y) for x, y in zip(names, selected_probs_list)]
    return res

In [19]:
project_name_hierarchical="subtask1_transformer_encoder_classification"

def return_trainer_hierarchical(model_name,unique_labels):
  api = wandb.Api()
  artifact=api.artifact(model_name)
  model_dir=artifact.download()
  if "persuasion_or_not" in model_name:
    lbls=unique_labels[0]
    id2label={0:lbls[0],1:lbls[1]}
    label2id={lbls[0]:0,lbls[1]:1}
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=len(unique_labels[0]),id2label=id2label, label2id=label2id)
  else:
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=len(unique_labels[0]), problem_type="multi_label_classification", ignore_mismatched_sizes=True)

  trainer = Trainer(model=model)
  if torch.cuda.is_available():
    trainer.model = model.cuda()
  return trainer

In [20]:
if train_set == 'train':
    prediction_set=dataset_train['train']
elif train_set == 'train_val':
    prediction_set=dataset_train_val['train_val']
elif train_set == 'train_dev':
    prediction_set=dataset_train_dev['train_dev']
else:
    prediction_set=dataset_val["val"]
prediction_set=prediction_set.remove_columns(["labels"])

In [21]:
prediction_set

Dataset({
    features: ['id', 'link', 'text'],
    num_rows: 8000
})

In [22]:
def get_new_labels(model_address,node,unique,threshold,binarizer):
  trainer=return_trainer_hierarchical(model_address,unique)
  print(curr_set)
  preds=get_preds_hierarchical(binarizer,trainer.predict(curr_set).predictions,threshold)

  for i in range(len(preds)):

    if len(preds[i][0])>0:

      t=prev_labels[i][0].index(node)
      _=prev_labels[i][1].pop(t)
      prev_labels[i][0].remove(node)
      parent_labels=prev_labels[i][0]
      parent_probs=prev_labels[i][1]

      new_labels.append((parent_labels+preds[i][0],parent_probs+preds[i][1]))
    else:
      new_labels.append(prev_labels[i])

In [23]:
model_nodes_1 = {
    "persuasion_or_not": "model-vinai_bertweet-large-persuasion_or_not-memes_0.5threshold_5e-06learningRate:v17",
    "persuasion": "model-vinai_bertweet-large-persuasion-memes_0.3threshold_5e-06learningRate:v10",
    "ethos": "model-vinai_bertweet-large-ethos-memes_0.35threshold_5e-06learningRate:v0",
    "pathos": "model-vinai_bertweet-large-pathos-memes_0.55threshold_5e-06learningRate:v0",
    "logos": "model-vinai_bertweet-large-logos-memes_0.25threshold_5e-06learningRate:v0",
    "ad_hominem": "model-vinai_bertweet-large-ad_hominem-memes_0.55threshold_5e-06learningRate:v0",
    "justification": "model-vinai_bertweet-large-justification-memes_0.25threshold_5e-06learningRate:v0",
    "reasoning": "model-vinai_bertweet-large-reasoning-memes_0.4threshold_5e-06learningRate:v2",
    "simplification": "model-vinai_bertweet-large-simplification-memes_0.45threshold_5e-06learningRate:v0",
    "distraction": "model-vinai_bertweet-large-distraction-memes_0.45threshold_5e-06learningRate:v0"
  }

threshold_nodes_1= {
    "persuasion_or_not": 0.5,
    "persuasion": 0.3,
    "ethos": 0.35,
    "pathos": 0.55,
    "logos": 0.25,
    "ad_hominem": 0.55,
    "justification": 0.25,
    "reasoning": 0.4,
    "simplification": 0.45,
    "distraction": 0.45
  }

Leaf nodes that have to entry edges have two probabilities

In [24]:
trainer=return_trainer_hierarchical(model_nodes_1["persuasion_or_not"],persuasion_or_not_unique)
threshold=threshold_nodes_1['persuasion_or_not']
prediction_set_tokenized=prediction_set.map(tokenize_hierarchical_1,batched=True)

preds_=get_preds_hierarchical(lb_persuasion_or_not,trainer.predict(prediction_set_tokenized).predictions,threshold)
preds=[el if el[0][0]=="Persuasion" else [] for el in preds_]
final_ds_hierarchical_train=dict(zip(prediction_set_tokenized["id"],preds))

model_seq=["Persuasion","Ethos","Pathos","Logos","Ad Hominem","Justification","Reasoning","Simplification","Distraction"]
node_names = ["persuasion","ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [persuasion_unique, ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
mlb_nodes = [mlb_persuasion, mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]

for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds_hierarchical_train.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds_hierarchical_train[x] for x in ids]
  new_labels=[]
  curr_set=prediction_set_tokenized.filter(lambda x: x["id"] in ids)
  
  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  get_new_labels(model_nodes_1[node_names[i]],node,unique_node_labels[i],threshold_nodes_1[node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds_hierarchical_train[ids[k]]=new_labels[k]

wandb: Downloading large artifact model-vinai_bertweet-large-persuasion_or_not-memes_0.5threshold_5e-06learningRate:v17, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:1.8


Node: Persuasion, curr_set: Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'attention_mask'],
    num_rows: 6853
})


wandb: Downloading large artifact model-vinai_bertweet-large-persuasion-memes_0.3threshold_5e-06learningRate:v10, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:2.9


Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'attention_mask'],
    num_rows: 6853
})


Filter:   0%|          | 0/8000 [00:00<?, ? examples/s]

...

Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'attention_mask'],
    num_rows: 2164
})


Filter:   0%|          | 0/8000 [00:00<?, ? examples/s]

Node: Distraction, curr_set: Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'attention_mask'],
    num_rows: 593
})


wandb: Downloading large artifact model-vinai_bertweet-large-distraction-memes_0.45threshold_5e-06learningRate:v0, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:2.8


Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'attention_mask'],
    num_rows: 593
})


In [25]:
final_ds_hierarchical_train

{'65635': (['Black-and-white Fallacy/Dictatorship'], [0.9995972514152527]),
 '67927': (['Glittering generalities (Virtue)', 'Loaded Language'],
  [0.9969383478164673, 0.999756395816803]),
 '68031': [],
 '77490': (['Repetition'], [0.9469527006149292]),
 '67641': [],
 '66402': [],
 '79204': (['Whataboutism', 'Whataboutism'],
  [0.9992213249206543, 0.9430420398712158]),
 '79372': (['Slogans'], [0.9703347682952881]),
 '68254': (['Loaded Language', 'Smears', 'Whataboutism'],
  [0.9998263716697693, 0.9998663663864136, 0.8870883584022522]),
 '69640': (['Smears'], [0.9995219707489014]),
 '70817': (['Flag-waving', 'Flag-waving', 'Slogans'],
  [0.9973745346069336, 0.33624035120010376, 0.7354105710983276]),
 '76124': (['Exaggeration/Minimisation',
   'Loaded Language',
   'Name calling/Labeling',
   'Smears',
   'Causal Oversimplification'],
  [0.9997304081916809,
   0.9989542961120605,
   0.906256377696991,
   0.9998987913131714,
   0.9990171194076538]),
 '64479': (['Loaded Language',
   'Name c

In [26]:
one_hot_hierarchical_train_1 = {
    id: (mlb_extended_nodes.transform([preds[0]])[0] if preds else mlb_extended_nodes.transform([[]])[0])
    for id, preds in final_ds_hierarchical_train.items()
}
one_hot_hierarchical_train_1

{'65635': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 '67927': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 '68031': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 '77490': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0]),
 '67641': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 '66402': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 '79204': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1]),
 '79372': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0]),
 '68254': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
     

### Hierarchical 2

In [27]:
#checkpoint_hierarchical_1="vinai/bertweet-large"
checkpoint_hierarchical_2 = "microsoft/deberta-v3-large"

In [28]:
tokenizer_hierarchical_2 = AutoTokenizer.from_pretrained(checkpoint_hierarchical_2)
data_collator_hierarchical_2 = DataCollatorWithPadding(tokenizer=tokenizer_hierarchical_2)
def tokenize_hierarchical_2(examples):
    encoding = tokenizer_hierarchical_2(examples["text"], truncation=True ,padding='max_length', max_length=512)
    return encoding

/home/nlp-lab-ws23/nlp_praktikum/nlp_praktikum_env/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [29]:
if train_set == 'train':
    prediction_set=dataset_train['train']
elif train_set == 'train_val':
    prediction_set=dataset_train_val['train_val']
elif train_set == 'train_dev':
    prediction_set=dataset_train_dev['train_dev']
else:
    prediction_set=dataset_val["val"]
prediction_set=prediction_set.remove_columns(["labels"])

In [30]:
model_nodes_2 = {
    "persuasion_or_not": "model-microsoft_deberta-v3-large-persuasion_or_not-memes_0.5threshold_5e-06learningRate:v9",
    "persuasion": "model-microsoft_deberta-v3-large-persuasion-memes_0.35threshold_5e-06learningRate:v0",
    "ethos": "model-microsoft_deberta-v3-large-ethos-memes_0.4threshold_5e-06learningRate:v8",
    "pathos": "model-microsoft_deberta-v3-large-pathos-memes_0.7threshold_5e-06learningRate:v0",
    "logos": "model-microsoft_deberta-v3-large-logos-memes_0.4threshold_5e-06learningRate:v6",
    "ad_hominem": "model-microsoft_deberta-v3-large-ad_hominem-memes_0.35threshold_5e-06learningRate:v0",
    "justification": "model-microsoft_deberta-v3-large-justification-memes_0.3threshold_5e-06learningRate:v1",
    "reasoning": "model-microsoft_deberta-v3-large-reasoning-memes_0.45threshold_5e-06learningRate:v0",
    "simplification": "model-microsoft_deberta-v3-large-simplification-memes_0.5threshold_5e-06learningRate:v1",
    "distraction": "model-microsoft_deberta-v3-large-distraction-memes_0.5threshold_5e-06learningRate:v1"
  }

threshold_nodes_2= {
    "persuasion_or_not": 0.5,
    "persuasion": 0.35,
    "ethos": 0.4,
    "pathos": 0.7,
    "logos": 0.4,
    "ad_hominem": 0.35,
    "justification": 0.3,
    "reasoning": 0.45,
    "simplification": 0.5,
    "distraction": 0.5
  }

In [31]:
trainer=return_trainer_hierarchical(model_nodes_2["persuasion_or_not"],persuasion_or_not_unique)
threshold=threshold_nodes_2['persuasion_or_not']
prediction_set_tokenized=prediction_set.map(tokenize_hierarchical_2,batched=True)

preds_=get_preds_hierarchical(lb_persuasion_or_not,trainer.predict(prediction_set_tokenized).predictions,threshold)
preds=[el if el[0][0]=="Persuasion" else [] for el in preds_]
final_ds_hierarchical_train=dict(zip(prediction_set_tokenized["id"],preds))

model_seq=["Persuasion","Ethos","Pathos","Logos","Ad Hominem","Justification","Reasoning","Simplification","Distraction"]
node_names = ["persuasion","ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [persuasion_unique, ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
mlb_nodes = [mlb_persuasion, mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]

for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds_hierarchical_train.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds_hierarchical_train[x] for x in ids]
  new_labels=[]
  curr_set=prediction_set_tokenized.filter(lambda x: x["id"] in ids)
  
  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  get_new_labels(model_nodes_2[node_names[i]],node,unique_node_labels[i],threshold_nodes_2[node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds_hierarchical_train[ids[k]]=new_labels[k]

wandb: Downloading large artifact model-microsoft_deberta-v3-large-persuasion_or_not-memes_0.5threshold_5e-06learningRate:v9, 1670.30MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:3.6


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8000 [00:00<?, ? examples/s]

Node: Persuasion, curr_set: Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6592
})


wandb: Downloading large artifact model-microsoft_deberta-v3-large-persuasion-memes_0.35threshold_5e-06learningRate:v0, 1670.30MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:3.8


...

Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1666
})


Filter:   0%|          | 0/8000 [00:00<?, ? examples/s]

Node: Distraction, curr_set: Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 447
})


wandb: Downloading large artifact model-microsoft_deberta-v3-large-distraction-memes_0.5threshold_5e-06learningRate:v1, 1670.30MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:0.8


Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 447
})


In [32]:
one_hot_hierarchical_train_2 = {
    id: (mlb_extended_nodes.transform([preds[0]])[0] if preds else mlb_extended_nodes.transform([[]])[0])
    for id, preds in final_ds_hierarchical_train.items()
}
one_hot_hierarchical_train_2

{'65635': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 '67927': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 '68031': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 '77490': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 '67641': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0]),
 '66402': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 '79204': array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1]),
 '79372': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0]),
 '68254': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
     

## Meta-Model

### Training

Concatenate predictions to one feature set

In [33]:
keys = one_hot_hierarchical_train_1.keys()
final_one_hot_hierarchical_train_1 = np.array([one_hot_hierarchical_train_1[key] for key in keys])
final_one_hot_hierarchical_train_2 = np.array([one_hot_hierarchical_train_2[key] for key in keys])
combined_features_train= np.concatenate((final_one_hot_hierarchical_train_1, final_one_hot_hierarchical_train_2), axis=1)
combined_features_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Prepare Gold Labels

In [34]:
if train_set == 'train':
    validation_set_df = pd.read_json(train_path)
elif train_set == 'train_val':
    validation_set_df= pd.read_json(train_val_path)
elif train_set == 'train_dev':
    validation_set_df= pd.read_json(train_dev_path)
else:
    validation_set_df = pd.read_json(val_path)
validation_set_df

,id,text,labels,link
0,65635,THIS IS WHY YOU NEED\n\nA SHARPIE WITH YOU AT ...,[Black-and-white Fallacy/Dictatorship],https://www.facebook.com/photo/?fbid=402355213...
1,67927,GOOD NEWS!\n\nNAZANIN ZAGHARI-RATCLIFFE AND AN...,"[Loaded Language, Glittering generalities (Vir...",https://www.facebook.com/amnesty/photos/531198...
2,68031,PAING PHYO MIN IS FREE!,[],https://www.facebook.com/amnesty/photos/427419...
3,77490,Move your ships away!\n\noooook\n\nMove your s...,[],https://www.facebook.com/rightpatriots/photos/...
4,67641,"WHEN YOU'RE THE FBI, THEY LET YOU DO IT.",[Thought-terminating cliché],https://www.facebook.com/AddictingInfoOrg/phot...
...,...,...,...,...
7995,70426,CATHING MEXICANS!!!,[Loaded Language],https://www.facebook.com/GovLookout/photos/pb....
7996,64176,\I WILL NEVER REGRET MY SUPPORT FOR PRESIDENT ...,"[Smears, Appeal to authority, Glittering gener...",https://www.facebook.com/SilentmajorityDJT/pho...
7997,68485,DEMAND KING SALMAN RELEASE CLERIC SHEIKH SALMA...,[Black-and-white Fallacy/Dictatorship],https://www.facebook.com/amnesty/photos/367919...
7998,75006,"\LISTEN TO MY ORDERS, EVIL PUPPET...\n\n... I ...","[Smears, Name calling/Labeling, Loaded Language]",https://www.facebook.com/photo/?fbid=234964353...


In [35]:
gold_labels_bin = mlb_extended_nodes.transform(validation_set_df['labels'])
gold_labels_bin

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Hyperparam Search

In [36]:
#summary_dir_path = folder_name + "subtask1/transformer/summaries/ensembling/summary_ensembling_train_val_1_" + model_name.replace("/","_") + "/"

In [37]:
def write_json(path,data,test=False):
  if not isinstance(data, dict):
    data = data.to_dict("records")

  with open(path, "w") as output_file:
      json.dump(data, output_file, indent=2,ensure_ascii=False)

def hierarchical_scores(gold_label_path, pred_label_path):
    scorer = folder_name + "subtask1/subtask_1_2a.py"
    command = f'python3 {scorer} --gold_file_path {gold_label_path} --pred_file_path {pred_label_path}'

    result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True)
    output = result.stdout.strip()

    parts = output.split('\t')
    f1_h = parts[0].split('=')[1]
    prec_h = parts[1].split('=')[1]
    rec_h = parts[2].split('=')[1]

    f1_h = float(f1_h)
    prec_h = float(prec_h)
    rec_h = float(rec_h)

    hierarchical_metrics = {"f1_hierarchical": f1_h, "precision_hierarchical": prec_h, "recall_hierarchical": rec_h}
    return hierarchical_metrics

def compute_metrics(estimator, x, y):
    y_pred = estimator.predict(x)
    labels= y
    print(f"y_pred: {y_pred}")
    print(f"labels: {labels}")
    preds = mlb_extended_nodes.inverse_transform(y_pred)
    pred_keys= [i for i in range(len(preds))]
    final_ds_pred_search=dict(zip(pred_keys,[list(pred) for pred in preds]))
    pred_df=pd.DataFrame.from_dict({"id":final_ds_pred_search.keys(),"labels":final_ds_pred_search.values()})
    final_ds_true_search=dict(zip(pred_keys,[list(label) for label in labels]))
    true_df=pd.DataFrame.from_dict({"id":final_ds_true_search.keys(),"labels":final_ds_true_search.values()})

    search_tmp_pred=summary_dir_path + "tmp/search_tmp_pred.json"
    if not os.path.exists(summary_dir_path):
        os.makedirs(summary_dir_path)

    write_json(search_tmp_pred,pred_df)

    search_tmp_true=summary_dir_path + "tmp/search_tmp_true.json"
    write_json(search_tmp_true,true_df)

    scores = hierarchical_scores(search_tmp_true,search_tmp_pred)
    print(f"Scores: {scores}")

    return scores['f1_hierarchical']

In [38]:
def search_random_forest():
    params = {
        "n_estimators": [300,500, 1000],
        #"max_depth": [30, 40, 50, 75, 100],
        "min_samples_leaf":  [1e-08,1e-07,1e-06],#[1e-08, 1e-07, 1e-06, 1e-05],
        "max_features":  [0.7,0.8,0.9],#[0.7, 0.8, 0.9, 1.0],
    }
    model = RandomForestClassifier()
    search = GridSearchCV(
        model, params, cv=2, n_jobs=-1, verbose=10, scoring="f1_weighted"
    )
    return search

search = search_random_forest()
search = search.fit(combined_features_train, gold_labels_bin)
meta_model = search.best_estimator_

Fitting 2 folds for each of 27 candidates, totalling 54 fits


/home/nlp-lab-ws23/nlp_praktikum/nlp_praktikum_env/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [39]:
import pickle
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)
with open(os.path.join(summary_dir_path, 'cvresults.pkl'), "wb") as f:
    pickle.dump(search.cv_results_, f)

In [40]:
cv_results = pd.DataFrame(search.cv_results_)[['param_max_features', 'param_min_samples_leaf', 'param_n_estimators', 'mean_test_score', 'rank_test_score']]
cv_results

,param_max_features,param_min_samples_leaf,param_n_estimators,mean_test_score,rank_test_score
0,0.7,0.0,300,0.887550,3
1,0.7,0.0,500,0.887281,6
2,0.7,0.0,1000,0.887481,5
3,0.7,0.0,300,0.887232,9
4,0.7,0.0,500,0.887716,2
5,0.7,0.0,1000,0.887261,7
6,0.7,0.000001,300,0.886731,19
7,0.7,0.000001,500,0.887234,8
8,0.7,0.000001,1000,0.886980,15
9,0.8,0.0,300,0.887534,4


In [41]:
pd.set_option("display.max_rows", None)

In [42]:
cv_results.sort_values('rank_test_score')

,param_max_features,param_min_samples_leaf,param_n_estimators,mean_test_score,rank_test_score
10,0.8,0.0,500,0.887785,1
4,0.7,0.0,500,0.887716,2
0,0.7,0.0,300,0.887550,3
9,0.8,0.0,300,0.887534,4
2,0.7,0.0,1000,0.887481,5
1,0.7,0.0,500,0.887281,6
5,0.7,0.0,1000,0.887261,7
7,0.7,0.000001,500,0.887234,8
3,0.7,0.0,300,0.887232,9
11,0.8,0.0,1000,0.887179,10


### Prediction

Hierarchical 

In [43]:
prediction_set=dataset_val['val']
prediction_set=prediction_set.remove_columns(["labels"])

In [44]:
trainer=return_trainer_hierarchical(model_nodes_1["persuasion"],persuasion_unique)
threshold=threshold_nodes_1['persuasion']
prediction_set_tokenized=prediction_set.map(tokenize_hierarchical_1 ,batched=True)

preds=get_preds_hierarchical(mlb_persuasion,trainer.predict(prediction_set_tokenized).predictions,threshold)
final_ds_hierarchical_pred=dict(zip(prediction_set_tokenized["id"],preds))

model_seq=["Ethos","Pathos","Logos","Ad Hominem","Justification","Reasoning","Simplification","Distraction"]
node_names = ["ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
mlb_nodes = [mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]

for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds_hierarchical_pred.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds_hierarchical_pred[x] for x in ids]
  new_labels=[]
  curr_set=prediction_set_tokenized.filter(lambda x: x["id"] in ids)

  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  get_new_labels(model_nodes_1[node_names[i]],node,unique_node_labels[i],threshold_nodes_1[node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds_hierarchical_pred[ids[k]]=new_labels[k]

wandb: Downloading large artifact model-vinai_bertweet-large-persuasion-memes_0.3threshold_5e-06learningRate:v10, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:3.0


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Node: Ethos, curr_set: Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'attention_mask'],
    num_rows: 375
})


wandb: Downloading large artifact model-vinai_bertweet-large-ethos-memes_0.35threshold_5e-06learningRate:v0, 1358.87MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:3.0


...

Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'attention_mask'],
    num_rows: 176
})


Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Node: Distraction, curr_set: Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'attention_mask'],
    num_rows: 41
})


wandb: Downloading large artifact model-vinai_bertweet-large-distraction-memes_0.45threshold_5e-06learningRate:v0, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:2.0


Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'attention_mask'],
    num_rows: 41
})


In [45]:
one_hot_hierarchical_pred_1 = {
    id: (mlb_extended_nodes.transform([preds[0]])[0] if preds else mlb_extended_nodes.transform([[]]))
    for id, preds in final_ds_hierarchical_pred.items()
}
one_hot_hierarchical_pred_1

{'63135': array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 '78590': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0]),
 '65834': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0]),
 '67394': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0]),
 '67709': array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 0]),
 '66205': array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 '79029': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0]),
 '78559': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 0]),
 '77741': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     

Hierarchical 2

In [46]:
#dataset_test = dataset_test.map(create_parent_labels)
prediction_set=dataset_val['val']
prediction_set=prediction_set.remove_columns(["labels"])

In [47]:
trainer=return_trainer_hierarchical(model_nodes_2["persuasion"],persuasion_unique)
threshold=threshold_nodes_2['persuasion']
prediction_set_tokenized=prediction_set.map(tokenize_hierarchical_2 ,batched=True)

preds=get_preds_hierarchical(mlb_persuasion,trainer.predict(prediction_set_tokenized).predictions,threshold)
final_ds_hierarchical_pred=dict(zip(prediction_set_tokenized["id"],preds))

model_seq=["Ethos","Pathos","Logos","Ad Hominem","Justification","Reasoning","Simplification","Distraction"]
node_names = ["ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
mlb_nodes = [mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]

for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds_hierarchical_pred.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds_hierarchical_pred[x] for x in ids]
  new_labels=[]
  curr_set=prediction_set_tokenized.filter(lambda x: x["id"] in ids)

  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  get_new_labels(model_nodes_2[node_names[i]],node,unique_node_labels[i],threshold_nodes_2[node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds_hierarchical_pred[ids[k]]=new_labels[k]

wandb: Downloading large artifact model-microsoft_deberta-v3-large-persuasion-memes_0.35threshold_5e-06learningRate:v0, 1670.30MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:2.1


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Node: Ethos, curr_set: Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 386
})


wandb: Downloading large artifact model-microsoft_deberta-v3-large-ethos-memes_0.4threshold_5e-06learningRate:v8, 1670.30MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:2.1


...

Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 147
})


Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Node: Distraction, curr_set: Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 33
})


wandb: Downloading large artifact model-microsoft_deberta-v3-large-distraction-memes_0.5threshold_5e-06learningRate:v1, 1670.30MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:2.1


Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 33
})


In [48]:
one_hot_hierarchical_pred_2 = {
    id: (mlb_extended_nodes.transform([preds[0]])[0] if preds else mlb_extended_nodes.transform([[]]))
    for id, preds in final_ds_hierarchical_pred.items()
}
one_hot_hierarchical_pred_2

{'63135': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0]),
 '78590': array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 '65834': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0]),
 '67394': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0]),
 '67709': array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 0]),
 '66205': array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]),
 '79029': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0]),
 '78559': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0]),
 '77741': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     

Concatenate predictions

In [49]:
keys = one_hot_hierarchical_pred_1.keys()
final_one_hot_hierarchical_pred_1 = np.array([one_hot_hierarchical_pred_1[key] for key in keys])
final_one_hot_hierarchical_pred_2 = np.array([one_hot_hierarchical_pred_2[key] for key in keys])
combined_features_pred= np.concatenate((final_one_hot_hierarchical_pred_1, final_one_hot_hierarchical_pred_2), axis=1)
combined_features_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [50]:
# Predict on the test set
y_pred = meta_model.predict(combined_features_pred)

Get Labels

In [51]:
preds = mlb_extended_nodes.inverse_transform(y_pred)
final_ds_pred=dict(zip(prediction_set_tokenized["id"],[list(pred) for pred in preds]))
final_ds_pred

{'63135': ['Black-and-white Fallacy/Dictatorship', 'Slogans'],
 '78590': ['Exaggeration/Minimisation', 'Loaded Language'],
 '65834': ['Smears'],
 '67394': ['Name calling/Labeling', 'Smears'],
 '67709': ['Appeal to authority', 'Smears'],
 '66205': ['Appeal to authority',
  'Glittering generalities (Virtue)',
  'Loaded Language',
  'Name calling/Labeling'],
 '79029': ['Glittering generalities (Virtue)', 'Loaded Language'],
 '78559': ['Smears'],
 '77741': ['Glittering generalities (Virtue)'],
 '63599': ['Appeal to authority',
  'Name calling/Labeling',
  'Smears',
  'Whataboutism'],
 '77552': ['Appeal to authority',
  'Loaded Language',
  'Name calling/Labeling',
  'Smears'],
 '71273': ['Black-and-white Fallacy/Dictatorship', 'Loaded Language'],
 '65211': ['Name calling/Labeling', 'Smears'],
 '71723': ['Loaded Language',
  'Name calling/Labeling',
  'Smears',
  'Thought-terminating cliché'],
 '70725': ['Appeal to authority', 'Exaggeration/Minimisation', 'Flag-waving'],
 '65147': ['Loaded 

### Evaluate using scorer script

In [52]:
pred_df=pd.DataFrame.from_dict({"id":final_ds_pred.keys(),"labels":final_ds_pred.values()})

val_pred_file=summary_dir_path + "val_pred.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)

write_json(val_pred_file,pred_df)

scores = hierarchical_scores(val_path,val_pred_file)
scores

{'f1_hierarchical': 0.67373,
 'precision_hierarchical': 0.61989,
 'recall_hierarchical': 0.73783}

### Create Dev Predictions

Hierarchical 1

In [53]:
prediction_set=dataset_test['test']

ValueError: Column name labels not in the dataset. Current columns in the dataset: ['id', 'text']

In [ ]:
trainer=return_trainer_hierarchical(model_nodes_1["persuasion"],persuasion_unique)
threshold=threshold_nodes_1['persuasion']
prediction_set_tokenized=prediction_set.map(tokenize_hierarchical_1 ,batched=True)

preds=get_preds_hierarchical(mlb_persuasion,trainer.predict(prediction_set_tokenized).predictions,threshold)
final_ds_hierarchical_pred=dict(zip(prediction_set_tokenized["id"],preds))

model_seq=["Ethos","Pathos","Logos","Ad Hominem","Justification","Reasoning","Simplification","Distraction"]
node_names = ["ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
mlb_nodes = [mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]

for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds_hierarchical_pred.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds_hierarchical_pred[x] for x in ids]
  new_labels=[]
  curr_set=prediction_set_tokenized.filter(lambda x: x["id"] in ids)

  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  get_new_labels(model_nodes_1[node_names[i]],node,unique_node_labels[i],threshold_nodes_1[node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds_hierarchical_pred[ids[k]]=new_labels[k]

In [ ]:
one_hot_hierarchical_pred_1 = {
    id: (mlb_extended_nodes.transform([preds[0]])[0] if preds else mlb_extended_nodes.transform([[]]))
    for id, preds in final_ds_hierarchical_pred.items()
}
one_hot_hierarchical_pred_1

Hierarchical 2

In [ ]:
prediction_set=dataset_val['val']
prediction_set=prediction_set.remove_columns(["labels"])

In [ ]:
trainer=return_trainer_hierarchical(model_nodes_2["persuasion"],persuasion_unique)
threshold=threshold_nodes_2['persuasion']
prediction_set_tokenized=prediction_set.map(tokenize_hierarchical_2 ,batched=True)

preds=get_preds_hierarchical(mlb_persuasion,trainer.predict(prediction_set_tokenized).predictions,threshold)
final_ds_hierarchical_pred=dict(zip(prediction_set_tokenized["id"],preds))

model_seq=["Ethos","Pathos","Logos","Ad Hominem","Justification","Reasoning","Simplification","Distraction"]
node_names = ["ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
mlb_nodes = [mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]

for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds_hierarchical_pred.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds_hierarchical_pred[x] for x in ids]
  new_labels=[]
  curr_set=prediction_set_tokenized.filter(lambda x: x["id"] in ids)

  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  get_new_labels(model_nodes_2[node_names[i]],node,unique_node_labels[i],threshold_nodes_2[node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds_hierarchical_pred[ids[k]]=new_labels[k]

In [ ]:
one_hot_hierarchical_pred_2 = {
    id: (mlb_extended_nodes.transform([preds[0]])[0] if preds else mlb_extended_nodes.transform([[]]))
    for id, preds in final_ds_hierarchical_pred.items()
}
one_hot_hierarchical_pred_2

In [ ]:
keys = one_hot_hierarchical_pred_1.keys()
final_one_hot_hierarchical_pred_1 = np.array([one_hot_hierarchical_pred_1[key] for key in keys])
final_one_hot_hierarchical_pred_2 = np.array([one_hot_hierarchical_pred_2[key] for key in keys])
combined_features_pred= np.concatenate((final_one_hot_hierarchical_pred_1, final_one_hot_hierarchical_pred_2), axis=1)
combined_features_pred

In [ ]:
# Predict on the test set
y_pred = meta_model.predict(combined_features_pred)

Get Labels

In [ ]:
preds = mlb_extended_nodes.inverse_transform(y_pred)
final_ds_pred=dict(zip(prediction_set_tokenized["id"],[list(pred) for pred in preds]))
final_ds_pred

In [ ]:
pred_df=pd.DataFrame.from_dict({"id":final_ds_pred.keys(),"labels":final_ds_pred.values()})

test_pred_file=summary_dir_path + "test_pred.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)

write_json(test_pred_file,pred_df)

Delete tmp files

In [ ]:
def delete_dir(path):
    if os.path.exists(path):
        try:
            shutil.rmtree(path)
            print(f"The directory {path} and all its contents have been deleted successfully")
        except OSError as e:
            print(f"Error: {e.strerror}")
    else:
        print(f"The directory {path} does not exist")

In [ ]:
def delete_output_dirs(parent_directory):
    for entry in os.listdir(parent_directory):
        entry_path = os.path.join(parent_directory, entry)

        if os.path.isdir(entry_path) and entry.startswith('output_'):
            delete_dir(entry_path)

In [ ]:
summary = {
    "checkpoint_hierarchical" : checkpoint_hierarchical_1,
    "checkpoint_hierarchical_2" : checkpoint_hierarchical_2,
    "training_set": train_set,
    "empty_labels": empty_labels,
    "best_models_hierarchical": model_nodes_1,
    "best_models_hierarchical_2": model_nodes_2,
    "meta_model": str(meta_model),
    "hierarchical_scores": scores
}

summary

In [ ]:
summary_file_path = summary_dir_path + "summary.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)
write_json(summary_file_path, summary)

In [ ]:
path = folder_name + "subtask1"
delete_output_dirs(path)
path = path + "/transformer/ensembling/"
delete_dir(path + "tumnlp")
delete_dir(path + "wandb")
delete_dir(path + "artifacts")
delete_dir(path + "tmp_trainer")
path = summary_dir_path + "tmp"
